In [5]:
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
warnings.filterwarnings("ignore", message=r"Implicit", category=UserWarning)
warnings.filterwarnings("ignore", message=r"cannot", category=UserWarning)

In [6]:
import os
import sys
sys.path.append('..')

from carla.recourse_methods import Dice, Wachter
from copy import deepcopy
from model_shifts import (DynamicCsvCatalog,
                          DynamicOnlineCatalog,
                          RecourseExperiment,
                          RecourseGenerator,
                          train_model,
                          plot_experiment,
                          generate_gif)

os.environ["CUDA_VISIBLE_DEVICES"] = ""

#     torch.manual_seed(0)
#     random.seed(0)
#     np.random.seed(0)

In [7]:
# Load the dataset from a csv file
# TODO: Iris? Penguins? Breast Cancer (categorical)?
dataset = DynamicCsvCatalog(file_path="../datasets/linearly_separable.csv", 
                            continuous=['feature1', 'feature2'], categorical=[],
                            immutables=[], target='target', test_size=0.3)
# dataset = DynamicOnlineCatalog("compas", 'score')

In [8]:
hyper_params = {'learning_rate': 0.01, 'epochs': 10, 'batch_size': 1, 'hidden_size': [5]}

# Recourse generated by DICE is compared with the Wachter generator, as they may modify data differently
# we need to keep track of two models and two datasets and update them independently
model = train_model(dataset, hyper_params)

generators = [
              RecourseGenerator('DICE_1', deepcopy(dataset), deepcopy(model),
                                Dice, {"num": 1}, hyper_params, 10),
            #   RecourseGenerator('DICE_2', deepcopy(dataset), deepcopy(model),
            #                     Dice, {"num": 2}, hyper_params, 30),
              RecourseGenerator('Wachter', deepcopy(dataset), deepcopy(model),
                                Wachter, {"loss_type": "BCE", "t_max_min": (1 / 2)}, hyper_params, 30)
             ]

# SET NAME BEFORE THE EXPERIMENT
name='test'
experiment = RecourseExperiment(dataset, model, generators, name, hyper_params)
experiment.run(epochs=20, recourse_per_epoch=1, calculate_p=1000)
experiment.save_data()

In [ ]:
experiment_path = f'../experiment_data/{experiment.experiment_name}'
for g in generators:
    generate_gif(experiment_path, g.name)

In [ ]:
config = [
    {'type': 'pos_MMD', 'dict_path': ['MMD', 'positive', 'value']},
    {'type': 'neg_MMD', 'dict_path': ['MMD', 'negative', 'value']},
    {'type': 'pos_distance', 'dict_path': ['decisiveness', 'positive']},
    {'type': 'neg_distance', 'dict_path': ['decisiveness', 'negative']},
    {'type': 'disagreement', 'dict_path': ['disagreement']},
    {'type': 'model_MMD', 'dict_path': ['model_MMD', 'value']}
]

for c in config:
    plot_experiment(experiment_path,
                    [g.name for g in generators],
                    c['type'],
                    c['dict_path'],
                    show_plot=False)

..\model_shifts\plotting\plot_experiment.py:77: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
  plt.ylim([0 - 0.2 * max(result), 1.2 * max(result)])
